Pre-processing YOLO + Manual

In [ ]:
import cv2
import pickle
import os
from ultralytics import YOLO

# === Constants ===
width = 130
height = 65
save_dir = 'cropped_img'
image_path = 'car1.png'
model_path = 'model/weights/best.pt'

# Create cropped image save directory if it doesn't exist
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# === Load and Resize Image ===
image = cv2.imread(image_path)
image = cv2.resize(image, (1280, 720))
base_image = image.copy()

# === Load YOLOv8 Model ===
model = YOLO(model_path)

# === Run YOLOv8 Inference ===
results = model.predict(source=image, conf=0.25, show_labels=False, show_conf=True)
detections = results[0].boxes.xyxy.cpu().numpy()

# === Build Detected Parking Spot List ===
positionList = []
for box in detections:
    x1, y1, x2, y2 = box
    x1, y1 = int(x1), int(y1)
    positionList.append(((x1, y1), len(positionList) + 1))

# === Save Cropped Images and Save Positions to Pickle ===
def save_cropped_img(img, pos, number):
    x, y = pos
    cropped_img = img[y:y+height, x:x+width]
    path = os.path.join(save_dir, f'roi_{number}.png')
    cv2.imwrite(path, cropped_img)
    print(f"✅ Saved cropped image for spot {number}: {path}")

for pos, number in positionList:
    save_cropped_img(image, pos, number)

with open('carposition.pkl', 'wb') as f:
    pickle.dump(positionList, f)

print("✅ YOLO detection complete and saved.")

# === Manual Edit Mode ===
selected_pos = None

def mouseclick(event, x, y, flags, param):
    global selected_pos, positionList

    if event == cv2.EVENT_LBUTTONDOWN:
        selected_pos = (x, y)

    elif event == cv2.EVENT_RBUTTONDOWN:
        for i, item in enumerate(positionList):
            pos, number = item
            x1, y1 = pos
            if x1 < x < x1 + width and y1 < y < y1 + height:
                print(f"🗑️ Removed spot {number} at {pos}")
                positionList.pop(i)
                with open('carposition.pkl', 'wb') as f:
                    pickle.dump(positionList, f)
                break

def get_number_input(image, pos):
    input_str = ""
    while True:
        temp_img = image.copy()
        cv2.putText(temp_img, f"Enter number: {input_str}", (pos[0], pos[1] - 20),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
        cv2.imshow("Edit Mode", temp_img)
        key = cv2.waitKey(0)

        if key in range(48, 58):  # 0–9
            input_str += chr(key)
        elif key == 8:  # Backspace
            input_str = input_str[:-1]
        elif key in [13, 10]:  # Enter
            return int(input_str) if input_str.isdigit() else None
        elif key == 27:  # ESC
            return None

# === Manual Editing Loop ===
while True:
    image = base_image.copy()

    for pos, number in positionList:
        x, y = pos
        cv2.rectangle(image, (x, y), (x + width, y + height), (255, 0, 255), 2)
        cv2.putText(image, str(number), (x, y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 255), 2)

    if selected_pos:
        new_number = get_number_input(image, selected_pos)
        if new_number is not None:
            positionList.append((selected_pos, new_number))
            save_cropped_img(base_image, selected_pos, new_number)
            with open('carposition.pkl', 'wb') as f:
                pickle.dump(positionList, f)
        selected_pos = None

    cv2.imshow("Edit Mode", image)
    cv2.setMouseCallback("Edit Mode", mouseclick)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()



0: 384x640 55 parking_spots, 29.7ms
Speed: 3.1ms preprocess, 29.7ms inference, 2.8ms postprocess per image at shape (1, 3, 384, 640)
✅ Saved cropped image for spot 1: cropped_img\roi_1.png
✅ Saved cropped image for spot 2: cropped_img\roi_2.png
✅ Saved cropped image for spot 3: cropped_img\roi_3.png
✅ Saved cropped image for spot 4: cropped_img\roi_4.png
✅ Saved cropped image for spot 5: cropped_img\roi_5.png
✅ Saved cropped image for spot 6: cropped_img\roi_6.png
✅ Saved cropped image for spot 7: cropped_img\roi_7.png
✅ Saved cropped image for spot 8: cropped_img\roi_8.png
✅ Saved cropped image for spot 9: cropped_img\roi_9.png
✅ Saved cropped image for spot 10: cropped_img\roi_10.png
✅ Saved cropped image for spot 11: cropped_img\roi_11.png
✅ Saved cropped image for spot 12: cropped_img\roi_12.png
✅ Saved cropped image for spot 13: cropped_img\roi_13.png
✅ Saved cropped image for spot 14: cropped_img\roi_14.png
✅ Saved cropped image for spot 15: cropped_img\roi_15.png
✅ Saved croppe

KeyboardInterrupt: 

: 